In [ ]:
# Necessary imports
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import pickle

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
%matplotlib inline

In [ ]:
# Load data
infile = open('whiskeys.pkl','rb')
my_list = pickle.load(infile)
infile.close()

## Data Cleaning and EDA

In [ ]:
df = pd.DataFrame(my_list)
df.head()

In [ ]:
df.shape

We can see that there are 1199 entries for the 11 columns we scraped for our data set, but let's also check for duplicates based on the name of the whiskey since other columns containing information on the brand, country, etc. are likely to have repeating values that aren't actually duplicates.

In [ ]:
# find the number of duplicate bottles
df.duplicated(subset='whiskey_name').sum()

In [ ]:
# find the duplicate value
df[df.duplicated(subset='whiskey_name') == True]

In [ ]:
# remove the duplicate
df.drop_duplicates(subset='whiskey_name', inplace=True)

In [ ]:
df.shape

In the `whiskey_name` column, we see that for whiskeys that include an apostrophe and an *s* in the title, the *s* is capitalized. We can change that so that the bottle name is titled correctly.

In [ ]:
df['whiskey_name'] = df['whiskey_name'].str.replace("'S", "'s")
df.head()

Next, we'll check for null values and drop any `NaN` values from columns that will later be continuous features.

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(subset=['user_rating', 'num_reviews', 'price'], inplace=True)
df.shape

We can also fill `NaN` values with 0 from columns that will later be categorical features.

In [ ]:
values = {'rating_source':0, 'state':0, 'taste':0}
df.fillna(value=values, inplace=True)
df.isnull().sum()

In [ ]:
df.head()

Now that our data is cleaned, we'll observe how the different features are related to one another, first in a heatmap and then in a pairplot.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(df.corr(), cmap="seismic", vmin=-1, vmax=1, annot=True, ax=ax);

In [ ]:
sns.pairplot(df, height=1.5, aspect=1);

We can see that our data has some outliers that may affect our model, specifically with respect to price. We'll first look at the average price of a whiskey and the standard deviation. After, we can sort the values and remove any outliers to see if we can improve the correlation between features.

In [ ]:
df.describe()

The average price for a bottle of whiskey is about \\$140 with a standard deviation of about $857. Looking below at the dataframe, it may be helpful to remove bottles with a value over \\$1000.

In [ ]:
df.sort_values(by=['price'], ascending=False).head(10)

In [ ]:
df.drop(df[df.price > 1000].index, inplace=True)
df.describe()

Now let's look at our plots again.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(df.corr(), cmap="seismic", vmin=-1, vmax=1, annot=True, ax=ax);
#plt.savefig('heatmap.svg')

In [ ]:
sns.pairplot(df, height=2, aspect=1);
#plt.savefig('pairplot.svg')

In [ ]:
df.describe()

In [ ]:
df1 = df
#df1['brand_counts'] = df.groupby(df.brand).transform('count')

In [ ]:
df1['brand_counts'] = df1.groupby('brand')['brand'].transform('count')

In [ ]:
df1.sort_values(by='brand_counts', ascending=False, inplace=True)
df1.head(30)

In [ ]:
g = sns.displot(data=df1, x="brand_counts", kde=True, color="red")
plt.xlabel("Bottles/Brand", size=20)
plt.ylabel("Count", size=20)
plt.title("Plot of Counts for Number of Bottles Produced Per Brand", size=24)
#g.set(xticks=[])
#g.set_xticklabels(rotation=45)
#plt.savefig('brand_frequency.svg')

#### Challenge 1
Build a linear model that uses only a constant term (a column of ones) to predict a continuous outcome (like domestic total gross). How can you interpret the results of this model? What does it predict? Make a plot of predictions against actual outcome. Make a histogram of residuals. How are the residuals distributed?

In [ ]:
df['target_mean'] = df['user_rating'].mean()
df.head()

In [ ]:
# Create an empty model
lr = LinearRegression()

# Choose the predictor variables, here all but the first which is the response variable
# This model is analogous to the Y ~ X1 + X2 + X3 + X4 + X5 + X6 model
X = df[['target_mean']]

# Choose the response variable(s)
y = df[['user_rating']]

# Fit the model to the full dataset
fit1 = lr.fit(X, y)

# Print out the R^2 for the model against the full dataset
lr.score(X,y)

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.scatter(y, lr.predict(X), color='r')
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')
ax.set_title('Scatter Plot of Predicted vs Actual')
plt.show()

In [ ]:
y_residuals = df[['user_rating']] - lr.predict(X)
sns.histplot(data=y_residuals)

#### Challenge 2
Repeat the process of challenge one, but also add one continuous (numeric) predictor variable. Also add plots of model prediction against your feature variable and residuals against feature variable. How can you interpret what's happening in the model?

In [ ]:
# Create an empty model
lr = LinearRegression()

# Choose the predictor variables, here all but the first which is the response variable
# This model is analogous to the Y ~ X1 + X2 + X3 + X4 + X5 + X6 model
X = df[['target_mean', 'price']]

# Choose the response variable(s)
y = df[['user_rating']]

# Fit the model to the full dataset
lr.fit(X, y)

# Print out the R^2 for the model against the full dataset
lr.score(X,y)

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.scatter(y, lr.predict(X), color='b')
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')
ax.set_title('Scatter Plot of Predicted vs Actual')
plt.show()

In [ ]:
y_residuals = df[['user_rating']] - lr.predict(X)
sns.histplot(data=y_residuals)

#### Challenge 3
Repeat the process of Challenge 1, but add a categorical feature (like genre). You'll have to convert a column of text into a number of numerical columns ("dummy variables"). How can you interpret what's happening in the model?

In [ ]:
# Choose the predictor variables, here all but the first which is the response variable
# This model is analogous to the Y ~ X1 + X2 + X3 + X4 + X5 + X6 model
X = df[['target_mean', 'price', 'country']]

# Choose the response variable(s)
y = df[['user_rating']]

In [ ]:
cat_variables = ['country']

X_cat = df[cat_variables]

In [ ]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False, drop='first')
ohe.fit(X_cat) 
cats = ohe.transform(X_cat)

In [ ]:
columns = ohe.get_feature_names(cat_variables)
X_cat_df = pd.DataFrame(cats, columns=columns, index=X_cat.index)
X_cat_df.head()

In [ ]:
X_cont = df[['target_mean', 'price']]

X_cont.head()

In [ ]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

ss.fit(X_cont)
X_scaled = ss.transform(X_cont)

cont_columns = X_cont.columns
X_scaled_df = pd.DataFrame(X_scaled, columns=cont_columns, index=X_cont.index)

X_scaled_df.head()

In [ ]:
X_combined = pd.concat([X_cat_df, X_scaled_df], axis='columns')

X_combined.head()

In [ ]:
lr = LinearRegression()

lr.fit(X_combined, y)

y_pred = lr.predict(X_combined)

y_pred

lr.score(X_combined, y)

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.scatter(y, y_pred, color='b')
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')
ax.set_title('Scatter Plot of Predicted vs Actual')
plt.show()

In [ ]:
y_residuals = y - y_pred
sns.histplot(data=y_residuals)

#### Challenge 4
Enhance your model further by adding more features and/or transforming existing features. Think about how you build the model matrix and how to interpret what the model is doing.

In [ ]:
# The first three predictor variables are continuous and the last three are categorical
X = df[['target_mean', 'num_reviews', 'price', 'brand', 'country', 'spirit_type']]

# Choose the response variable(s)
y = df[['user_rating']]

In [ ]:
cat_variables = ['brand', 'country', 'spirit_type']

X_cat = df[cat_variables]

In [ ]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False, drop='first')
ohe.fit(X_cat) 
cats = ohe.transform(X_cat)

In [ ]:
columns = ohe.get_feature_names(cat_variables)
X_cat_df = pd.DataFrame(cats, columns=columns, index=X_cat.index)
X_cat_df.head()

In [ ]:
X_cont = df[['target_mean', 'num_reviews', 'price']]

X_cont.head()

In [ ]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

ss.fit(X_cont)
X_scaled = ss.transform(X_cont)

cont_columns = X_cont.columns
X_scaled_df = pd.DataFrame(X_scaled, columns=cont_columns, index=X_cont.index)

X_scaled_df.head()

In [ ]:
X_combined = pd.concat([X_cat_df, X_scaled_df], axis='columns')

X_combined.head()

In [ ]:
lr = LinearRegression()

lr.fit(X_combined, y)

y_pred = lr.predict(X_combined)

y_pred

lr.score(X_combined, y)

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.scatter(y, y_pred, color='b')
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')
ax.set_title('Scatter Plot of Predicted vs Actual')
plt.show()

In [ ]:
y_residuals = y - y_pred
sns.histplot(data=y_residuals)

#### Challenge 5
Fitting and checking predictions on the exact same data set can be misleading. Divide your data into two sets: a training and a test set (roughly 75% training, 25% test is a fine split). Fit a model on the training set, check the predictions (by plotting versus actual values) in the test set.

In [ ]:
from sklearn.model_selection import train_test_split

# The first three predictor variables are continuous and the last three are categorical
X = df[['target_mean', 'num_reviews', 'price', 'brand', 'country', 'spirit_type']]

# Choose the response variable(s)
y = df[['user_rating']]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
cat_variables = ['brand', 'country', 'spirit_type']

X_train_cat = X_train[cat_variables]

In [ ]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
ohe.fit(X_train_cat) 
cats = ohe.transform(X_train_cat)

In [ ]:
columns = ohe.get_feature_names(cat_variables)
X_train_cat_df = pd.DataFrame(cats, columns=columns, index=X_train_cat.index)
X_train_cat_df.head()

In [ ]:
X_test_cat = X_test[['brand', 'country', 'spirit_type']]

cats_test = ohe.transform(X_test_cat) # REMEMBER ONLY TRANSFORM ON TEST SET

cat_columns = ohe.get_feature_names(['brand', 'country', 'spirit_type'])
X_test_cat_df = pd.DataFrame(cats_test, columns=cat_columns, index=X_test_cat.index)
X_test_cat_df.head()

In [ ]:
X_train_cont = X_train[['target_mean', 'num_reviews', 'price']]

X_train_cont.head()

In [ ]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

ss.fit(X_train_cont)
X_train_scaled = ss.transform(X_train_cont)

cont_columns = X_train_cont.columns
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=cont_columns, index=X_train_cont.index)

X_train_scaled_df.head()

In [ ]:
X_test_cont = X_test[['target_mean', 'num_reviews', 'price']]

X_test_cont.head()

In [ ]:
ss.fit(X_test_cont)

X_test_scaled = ss.transform(X_test_cont)

cont_columns = X_test_cont.columns
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=cont_columns, index=X_test_cont.index)

X_test_scaled_df.head()

In [ ]:
X_train_combined = pd.concat([X_train_cat_df, X_train_scaled_df], axis='columns')

X_train_combined.head()

In [ ]:
X_test_combined = pd.concat([X_test_cat_df, X_test_scaled_df], axis='columns')

X_test_combined.head()

In [ ]:
lr = LinearRegression()

lr.fit(X_train_combined, y_train)

y_pred = lr.predict(X_test_combined)

y_pred

In [ ]:
score = lr.score(X_test_combined, y_test) 
score

In [ ]:
lr.score(X_train_combined, y_train)

The score for the training set is greater than the test set, which means that the data is very overfit!

In [ ]:
feature_names = X_test_combined.columns

coefficient_values = np.round(lr.coef_, 3)

feats_values = list(zip(feature_names, coefficient_values)) 

sorted(feats_values, key=lambda x: x[1])

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.scatter(y_train, lr.predict(X_train_combined), color='b')
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')
ax.set_title('Scatter Plot of Predicted vs Actual')
plt.show()

#### LASSO

In [ ]:
#Separate our features from our target

X = df.loc[:,['num_reviews', 'price', 'brand', 'country', 'spirit_type']]

y = df['user_rating']

# create overall quality squared term, which we expect to 
# help based on the relationship we see in the pair plot 
X['price_squared'] = X['price'] ** 2

In [ ]:
X.info()

In [ ]:
#Split the data 60 - 20 - 20 train/val/test

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2,random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=.25, random_state=43)

In [ ]:
X_train.columns

In [ ]:
X_train.shape

In [ ]:
X_val.shape

In [ ]:
X_test.shape

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat_variables = ['brand', 'country', 'spirit_type']
X_train_cat = X_train[cat_variables]

ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
ohe.fit(X_train_cat) 
cats = ohe.transform(X_train_cat)

In [ ]:
columns = ohe.get_feature_names(cat_variables)
X_train_cat_df = pd.DataFrame(cats, columns=columns, index=X_train_cat.index)
X_train_cat_df.head()

In [ ]:
X_train_cat_df.shape

In [ ]:
X_test_cat = X_test[['brand', 'country', 'spirit_type']]

cats_test = ohe.transform(X_test_cat) # REMEMBER ONLY TRANSFORM ON TEST SET

cat_columns = ohe.get_feature_names(['brand', 'country', 'spirit_type'])
X_test_cat_df = pd.DataFrame(cats_test, columns=cat_columns, index=X_test_cat.index)
X_test_cat_df.head()

In [ ]:
X_val_cat = X_val[['brand', 'country', 'spirit_type']]

cats_val = ohe.transform(X_val_cat) # REMEMBER ONLY TRANSFORM ON TEST SET

cat_columns = ohe.get_feature_names(['brand', 'country', 'spirit_type'])
X_val_cat_df = pd.DataFrame(cats_val, columns=cat_columns, index=X_val_cat.index)
X_val_cat_df.head()

In [ ]:
X_train_cont = X_train[['num_reviews', 'price']]

X_train_cont.head()

In [ ]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

ss.fit(X_train_cont)
X_train_scaled = ss.transform(X_train_cont)

cont_columns = X_train_cont.columns
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=cont_columns, index=X_train_cont.index)

X_train_scaled_df.head()

In [ ]:
X_test_cont = X_test[['num_reviews', 'price']]

X_test_cont.head()

In [ ]:
X_test_scaled = ss.transform(X_test_cont)

cont_columns = X_test_cont.columns
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=cont_columns, index=X_test_cont.index)

X_test_scaled_df.head()

In [ ]:
X_val_cont = X_val[['num_reviews', 'price']]

X_val_cont.head()

In [ ]:
X_val_scaled = ss.transform(X_val_cont)

cont_columns = X_test_cont.columns
X_val_scaled_df = pd.DataFrame(X_val_scaled, columns=cont_columns, index=X_val_cont.index)

X_val_scaled_df.head()

In [ ]:
X_train_combined = pd.concat([X_train_cat_df, X_train_scaled_df], axis='columns')

X_train_combined.head()

X_train_combined.shape

In [ ]:
X_test_combined = pd.concat([X_test_cat_df, X_test_scaled_df], axis='columns')

X_test_combined.head()

In [ ]:
X_val_combined = pd.concat([X_val_cat_df, X_val_scaled_df], axis='columns')

X_val_combined.head()

In [ ]:
# Run the cross validation, find the best alpha, refit the model on all the data with that alpha

alphavec = 10**np.linspace(-2,2,200)

lasso_model = LassoCV(alphas = alphavec, cv=5)
lasso_model.fit(X_train_combined, y_train)

In [ ]:
# This is the best alpha value it found - not far from the value
# selected using simple validation
lasso_model.alpha_

In [ ]:
# These are the (standardized) coefficients found
# when it refit using that best alpha
list(zip(X_train.columns, lasso_model.coef_))

In [ ]:
# Make predictions on the test set using the new model
test_set_pred = lasso_model.predict(X_test_combined)

In [ ]:
lasso_model.score(X_train_combined, y_train)

In [ ]:
lasso_model.score(X_test_combined, y_test)

In [ ]:
df['brand'].value_counts().head(30)

In [ ]:
sns.jointplot(x="rating", y="user_rating", data=df, order=2, kind="reg");
plt.xlabel("Critic Rating", size=20)
plt.ylabel("User Rating", size=20)
plt.title("User Rating vs. Critic Rating", size=24, y=1.25)
#plt.savefig('user_rating_vs_rating.svg')

In [ ]:
g = sns.displot(df, x='country', binwidth=8, hue='country')
plt.xlabel("Country", size=20)
plt.ylabel("Count", size=20)
plt.title("Number of Whiskeys Manufactured by Country", size=24)
g.set_xticklabels(rotation=45)
#plt.savefig('whiskeys_by_country.svg')